In [17]:
# Using readlines()
file1 = open('dataset_tsmc2014\dataset_TSMC2014_NYC.txt', 'r')
file2 = open('dataset_tsmc2014\dataset_TSMC2014_TKY.txt', 'r')
with open('dataset_tsmc2014\dataset_TSMC2014_NYC.txt') as f:
   file1_lines = sum(1 for _ in f)
lines = file1.readlines() + file2.readlines()

count = 0
check_ins = {}
poi_coos = {}
poi_cat = {}
normalized_vid = {}
user_time_prefs_count = {}
users = []
num_venues = 0 

for line in lines:
    line_split = line.split()
    if count == file1_lines:
        users_file_1 = len(users)
    uid = int(line_split[0]) - 1 if count < file1_lines else int(line_split[0]) - 1 + users_file_1
    vid = line_split[1]
    cid = line_split[2]
    for i in range(4,9):
        if line_split[i].replace('.','',1).replace('-','',1).isdigit():
            lat = line_split[i]
            lng = line_split[i+1]
            break
    
    #coords,cat
    if vid not in normalized_vid:
        normalized_vid[vid] = vid = num_venues

        poi_coos[vid]= (lat,lng)
        poi_cat[vid]= cid
        num_venues +=1
    else:
        vid = normalized_vid[vid]

    #check ins
    if(uid,vid) in check_ins:
        check_ins[(uid,vid)] += 1
    else:
        check_ins[(uid,vid)] = 1

    if uid not in users:
        users.append(uid)
    
    totd = 'morning'
    if uid not in user_time_prefs_count:
        user_time_prefs_count[uid] = {}
    if totd not in user_time_prefs_count[uid]:
        user_time_prefs_count[uid][totd] = 1
    else:
        user_time_prefs_count[uid][totd] += 1
    if count == 200:
        break
    count+= 1
num_user = len(users)
#print(num_user, len(check_ins))
#print(user_time_prefs)

In [20]:
import operator
#max(stats.iteritems(), key=operator.itemgetter(1))[0]
user_time_prefs = {}
for i,s in user_time_prefs_count.items():
    user_time_prefs[i] = max(s.items(), key=operator.itemgetter(1))[0]



{469: 'morning', 978: 'morning', 68: 'morning', 394: 'morning', 86: 'morning', 483: 'morning', 641: 'morning', 291: 'morning', 427: 'morning', 876: 'morning', 624: 'morning', 690: 'morning', 115: 'morning', 930: 'morning', 52: 'morning', 444: 'morning', 134: 'morning', 894: 'morning', 703: 'morning', 890: 'morning', 283: 'morning', 900: 'morning', 802: 'morning', 698: 'morning', 99: 'morning', 442: 'morning', 688: 'morning', 389: 'morning', 31: 'morning', 229: 'morning', 833: 'morning', 961: 'morning', 1031: 'morning', 524: 'morning', 317: 'morning', 1046: 'morning', 737: 'morning', 363: 'morning', 397: 'morning', 838: 'morning', 423: 'morning', 750: 'morning', 888: 'morning', 611: 'morning', 642: 'morning', 717: 'morning', 767: 'morning', 14: 'morning', 457: 'morning', 279: 'morning', 1015: 'morning', 280: 'morning', 365: 'morning', 275: 'morning', 226: 'morning', 79: 'morning', 672: 'morning', 336: 'morning', 915: 'morning', 638: 'morning', 751: 'morning', 712: 'morning', 448: 'morni

In [12]:
import random
with open('2_MGMPFM/new_data/fsq_data_size.txt', 'w') as f:
    f.write(str(num_user) + ' ' + str(num_venues))

with open('2_MGMPFM/new_data/fsq_pois_coos.txt', 'w') as f:
    for i,s in poi_coos.items():
        f.write(str(i) + ' ' + s[0] + ' ' + s[1] + '\n')

with open('2_MGMPFM/new_data/fsq_pois_cats.txt', 'w') as f:
    for i,s in poi_cat.items():
        f.write(str(i) + ' ' + s + '\n')

l = list(check_ins.items())
random.shuffle(l)
check_ins = dict(l)
print(len(check_ins))
cnt = 0
with open('2_MGMPFM/new_data/fsq_test.txt', 'w') as f1:
    with open('2_MGMPFM/new_data/fsq_train.txt', 'w') as f2:
        with open('2_MGMPFM/new_data/fsq_tune.txt', 'w') as f3:
            for i,s in check_ins.items():
                if cnt< 250000:
                    f1.write(str(i[0]) + ' ' + str(i[1]) + ' ' + str(s) + '\n')
                elif cnt < 280000:
                    f2.write(str(i[0]) + ' ' + str(i[1]) + ' ' + str(s) + '\n')
                else:
                    f3.write(str(i[0]) + ' ' + str(i[1]) + ' ' + str(s) + '\n')
                cnt +=1



302979
